In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 17.3 MB/s 
     |████████████████████████████████| 448 kB 45.3 MB/s 


In [3]:
from konlpy.tag import Okt 
okt = Okt()

In [5]:
text = '안녕하세요 룰루룰라랄 무무ㅜ운'
okt.morphs(text)

['안녕하세요', '룰루', '룰라', '랄', '무무', 'ㅜ', '운']

# 댓글 수집

In [7]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=181414&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'

import requests 
from bs4 import BeautifulSoup

In [8]:
page = 1
response = requests.get(url.format(1))
response

<Response [200]>

In [9]:
response.text

'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t<meta charset="utf-8">\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\t<title>네이버 영화</title>\r\n\t\n\n\n\t\n\t\n\t\n\t\n\t\t<link rel="shortcut icon" href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" type="image/x-icon">\n\t\t\t\n\n\r\n\t\n<link rel="stylesheet" type="text/css" href="/css/common.css?20220127220412" />\r\n\t\n<link rel="stylesheet" type="text/css" href="/css/movie_tablet.css?20220127220412" />\n<link rel="stylesheet" type="text/css" href="/css/movie_end.css?20220127220412" />\r\n\t\r\n\t\n\n\n<script type="text/javascript" src="/js/deploy/movie.all.js?20220127220412"></script>\n\n\n\r\n</head>\r\n<body>\r\n\t<!-- content -->\r\n<input type="hidden" name="movieCode" id="movieCode" value="181414"/>\r\n<input type="hidden" name="onlyActualPointYn" id="onlyActualPointYn" value="N"/>\r\n<input type="hidden" name="includeSpoilerYn" id="includeSpoilerYn" value="N"/>\r\

In [13]:
soup = BeautifulSoup(response.text, 'html.parser')

In [28]:
review_box = soup.find("div", attrs={"class": "score_result"}).find_all('li')

for idx, box in enumerate(review_box):
  score = box.find("em").get_text().strip()
  review = box.find("span", attrs={"id": f"_filtered_ment_{idx}"}).get_text().strip()
  print(score)
  print(review)

8
빨간옷중.. 유일하게 언어를 쓸수 있는 이유..
10
공포영화인줄 알았는데 미친영화였다...
8
영화 초반=컨저링급 후반=짱구의 좌충우돌 아메리카 대모험
6
호불호 많이 갈릴듯 소재는 참신했으나 풀어나가는 능력이 다소 미흡했다. 겟아웃 정도의 참신함과 몰입력은 아닌거같다.
10
재밋게 봣는데 궁금한건 손에 손잡고 계속 잇는겨? 저거 하면 뭐 어케 되는건데
8
어떤 영화가 낫다 공감 안낫다 비공감 이런것좀 하지마 맨날 이런게 베댓이야 어스자체만의 평점을 매기라고
10
루피타 뇽 연기력 미쳤다...
5
겟아웃 보다 확실히 별로입니다.
2
재밌냐 이게 ? 솔직히 이게 재밌냐고
10
진짜 감독이 천재인듯함


# 댓글 수집 자동화

In [35]:
def get_reple(page = 1):
  response = requests.get(url.format(page))
  soup = BeautifulSoup(response.text, 'html.parser')

  s, t = [], [] # score와 text를 담을 리스트

  # 리뷰 모음 박스
  review_box = soup.find("div", attrs={"class": "score_result"}).find_all('li') 

  # 리뷰 별 평점/
  for idx, box in enumerate(review_box):
    score = box.find("em").get_text().strip()
    review = box.find("span", attrs={"id": f"_filtered_ment_{idx}"}).get_text().strip()

    if int(score) >= 8:
      s.append(1)
      t.append(review)
    elif int(score) <= 5:
      s.append(0)
      t.append(review)
  return s, t

In [41]:
import time
from tqdm import tqdm
score, text = [], []

for i in tqdm(range(1,10000)):
  time.sleep(1)
  s, t = get_reple(i)
  score += s
  text += t

100%|██████████| 9999/9999 [4:31:27<00:00,  1.63s/it]


In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd

df = pd.DataFrame([score, text]).T
df.columns = ['score','text']

In [56]:
df.to_csv('/content/drive/MyDrive/study/test.csv')

# train/test set 분리

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text, score ,test_size=0.2, random_state=0)

In [43]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

50218 50218
12555 12555


# TFIDF 벡터화

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9) # 최소 개수 3개, 90% 이상 나오는 단어는 사용X
tfv.fit(X_train)
tfv_X_train = tfv.transform(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


# 모델 학습

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [47]:
clf =LogisticRegression(random_state=0)
params ={'C': [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_X_train, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [48]:
grid_cv.best_params_

{'C': 7}

In [49]:
grid_cv.best_score_

0.9787327459036858

In [50]:
tfv_X_test = tfv.transform(X_test)
grid_cv.best_estimator_.score(tfv_X_test, y_test)

0.9792911190760654

In [52]:
a = ['진짜 너무 재미있어요 꼭 보세요', '시간이 아까운 영화', '이거 볼 바에 클레멘타인 보겠다', '형편없어요']
my_review = tfv.transform(a)
grid_cv.best_estimator_.predict(my_review)

array([1, 0, 1, 0])